In [73]:
from pandas import read_csv
import re

In [2]:
import spacy
from spacy.tokens import DocBin

nlp = spacy.load("de_core_news_sm")

In [74]:
from glob import glob
daten = glob("plenarprotokolle/testing_prep/*.xml.spacy")
satzzeichen = re.compile(r'.*[^a-zA-Z0-9\u0080-\u00FF#].*')

In [4]:
spacy_db = {}
for datei in daten:
    protokoll = DocBin(store_user_data=True).from_bytes(open(datei, "rb").read())
    protokoll = list(protokoll.get_docs(nlp.vocab))
    datei = datei.lstrip("plenarprotokolle/testing_prep/")
    spacy_db[datei] = protokoll

In [31]:
def collect_classifiers_sentiws(sourcefile):
    with open(sourcefile) as csv_file:
        classifiers = read_csv(csv_file, sep="\t", header=None, names=["lemma", "wert", "formen"])
        classifiers["formen"] = classifiers["formen"].astype(str)
        classifiers["formen"] = classifiers["formen"].apply(lambda x: x.split(","))
        classifiers[["lemma","pos"]] = classifiers["lemma"].str.split("|",expand=True)
        classifiers["lemma"] = classifiers["lemma"].astype(str)
        for formen, lemma in zip(classifiers.formen, classifiers.lemma):
            formen = formen.append(lemma)
    return classifiers

In [32]:
classifiers_neg = collect_classifiers_sentiws("SentiWS/SentiWS_v2.0_Negative.txt")
classifiers_pos = collect_classifiers_sentiws("SentiWS/SentiWS_v2.0_Positive.txt")

In [66]:
def collect_classifiers_gpc(sourcefile):
    with open(sourcefile) as csv_file:
        classifiers = read_csv(csv_file, sep="\t", header=None, names=["form","lemma","pos","polaritaet","probabilitaet","whatever"])
        classifiers = classifiers.drop(labels=["probabilitaet","whatever"], axis=1)
    return classifiers
        

In [67]:
classifiers_neg_gpc = collect_classifiers_gpc("gpc/GermanPolarityClues-Negative-21042012.tsv")
classifiers_pos_gpc = collect_classifiers_gpc("gpc/GermanPolarityClues-Positive-21042012.tsv")


In [95]:
for f, protokoll in spacy_db.items():
    for rede in protokoll:
        rede.user_data["entitaeten"] = [x.text for x in rede.ents]
        rede.user_data["entitaeten"] = [x for x in rede.user_data["entitaeten"] if not x == "||"]          
        

In [ ]:
'''
Für die Analyse:
GPC hat keine genauen Werte, nur "negative" und "positive".
Deswegen wird GPC nur genutzt, um zu bestimmen ob ein Abschnitt negativ oder positiv ist
SentiWS wird zusätzlich noch benutzt, um die Intensität zu messen.
'''

